# Tracking objects / pipeline design

It is important to be able to order the various image processing tools you have learnt in an automated pipeline so you don't have to apply the same N transformations over and over again to your dataset of M images. Today wou will learn how to build your very own image processing pipeline, along with some nifty video processing tools.

We will do this in the context of an image processing method known as "tracking". You already know how to identify separate objects in your images and measure their properties (position, size, intensity etc.). Now imagine you have a time-series of images (or a video...). Tracking is essentially the ability to identify the same object in your entire time-series consistently. We will also demonstrate how this process can fail, and what to watch out for when you are analyzing your own data.

### When do you start developing a pipeline?

A typical research plan that a student might present looks like 

Order supplies --> Build a device --> Collect some data --> Analyze said data --> ~~Profit~~ Publish

This process is a pipeline! But it's out of order. Let's find out how with a real-world example.

### A rat behavior experiment
A post-doc wants to do a rat behavior experiment. For the experiment to work, it's critical to know whether or not the rat is moving at any given point in time. Each experiment runs for over an hour, but our post-doc reasons that she can set up a video camera synchronized to the rest of the experimental equipment, record the rat moving around its habitat, and determine when motion happened later, perhaps by doing some image processing. A labmate suggests she attach a bright red light to the rat's head to make it easy to track by computer - just find the reddest part in the red channel at each frame! Our post-doc does this, then collects more than 30 hours worth of data over two weeks.

In [1]:
%matplotlib inline

To work with video data, have a new import today: `moviepy`. This is a library that helps you interact with movie files. Video file formats utilize lossy compression of some kind, but for something like a rat moving around, where we're not exactly trying to make quantitative measurements of photon counts, it's preferable to save space and take compression losses. Even for display in a paper's supplement, video formats like mp4 and mpeg are useful.

In [2]:
from moviepy import editor as mpy

In [3]:
vid_file_name = "../data/rattrack.mp4"
vid = mpy.VideoFileClip(vid_file_name)
mpy.ipython_display(vid, width=480)

How would you begin processing this data to track the rat? What's our strategy going to be? What's different about this data compared to what we've seen before?

Let's look at the data in detail.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
sns.set_style('dark')

To extract the frames in the video, we will use the `VideoFileClip.get_frame(t)` function or `VideFileClip.iter_frames()`.

In [6]:
vid.get_frame?

In [7]:
vid.iter_frames?

Let's start with the very first frame at time $t = 0$.

In [8]:
frame = vid.get_frame(0)

This is a video in color. How many dimensions do we expect this frame to have?

In [9]:
frame.shape

As a first step, let's see if we can in fact find a red dot in a still frame.

In [10]:
plt.imshow(frame)

And just the red channel?

In [11]:
red = frame[:,:,0]
plt.imshow(red)

This looks tractable! How might we grab the brightest point?

In [12]:
index = np.argmax(red)
print(index)

What does this number mean? It turns out for that multidimensional arrays, `argmax` just tells you the position of the maximum value _if the array had first been flattened_. The brilliant minds behind Numpy know this probably isn't what you want, so they've provided you with `numpy.unravel_index`. It just needs to know the original array shape.

In [13]:
row_index, column_index = np.unravel_index(index, red.shape)

It turns out we can help ourselves visualize that point by plotting directly on top of an image we made with imshow. `plt.plot` takes a list of $x$, $y$ values, and a style parameter (e.g. 'o' makes the points little circles) - it works exactly like Matlab's `plot` if you're familiar with that.

**Exercise** how would you change the color of this pixel to make it stand out in the original color image?

In [14]:
plt.imshow(frame)
plt.plot(column_index, row_index, 'o')

# Fix up the margins - plotting expands the plot area
plt.xlim([0, frame.shape[1]])
plt.ylim([frame.shape[0], 0])

In [15]:
# smooth
# argmax

Now what?

We need to do this for every frame in the video.

In [16]:
num_frames = vid.duration * vid.fps

# a time x dim table. dim = 2 since we have 2D frames
dot_position = np.zeros((num_frames, 2))

for frame_num, frame in enumerate(vid.iter_frames()):
    red = frame[:,:,0]
    smooth = red# smoothing
    peak = np.argmax(smooth)
    position = np.unravel_index(peak, red.shape)
    dot_position[frame_num, :] = position # set the whole row of the table
    

**Exercise** How might you visualize how the rat is moving with time?

There are two approaches to solving this: look at x and y position independently against time, or look at the path the animal takes through time. Both are shown below.

In [17]:
plt.plot(dot_position)

In [18]:
plt.plot(dot_position[:,0], dot_position[:,1])

Look at those jumps! We have a teleporting rat! Let's look at a time where the rat seems to teleport. Surely we are zeroing in on a major scientific discovery.

In [19]:
jump_size = np.zeros(num_frames - 1)

for t in range(0, int(num_frames - 1)):
    jump_size[t] = np.sqrt(np.sum(np.square(dot_position[t+1,:] - dot_position[t,:])))
    
biggest_jump_t = np.argmax(jump_size) + 1

In [22]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
before = biggest_jump_t - 1
after = biggest_jump_t
ax[0].plot(dot_position[before,1], dot_position[before,0], 'o', fillstyle='none', markersize=20, markeredgewidth=5)
ax[0].imshow(vid.get_frame((before)/ vid.fps))
ax[1].plot(dot_position[after,1], dot_position[after,0], 'o', fillstyle='none', markersize=20, markeredgewidth=5)
ax[1].imshow(vid.get_frame(after / vid.fps))

In [23]:
print("Time of biggest jump: {} seconds".format(biggest_jump_t / vid.fps))

So what is happening at that time?

In [24]:
toi = biggest_jump_t / vid.fps # TOI = time of interest
short_clip = vid.subclip(toi - 1, toi + 2)
mpy.ipython_display(short_clip, width=480, loop=1)

The problem seems to be that the animal has room to turn its head and occulude the light with its head or the wires it's attached to. In the period where the light is occluded we need to either guess that it doesn't move until we find it again, or use some other tracking mechanism. It's also potentially hard to tell when we lose the light - right now we just look for the brighest red pixel, which will always exist. In fact, a white pixel looks pretty bright in the red channel.

Although there are some improvements we can make, it turns out that this problem went from very easy to very hard.

That's because the real problem occured weeks ago, here: "Collect some data --> Analyze said data". Collecting and analyzing data are not separable steps. You should be building your data processing pipeline while you're building your experiment, and iterating on it as you're collecting data.